In [7]:
import os
import json
import pandas as pd
import string
import re
from collections import Counter
from konlpy.tag import Okt
import numpy as np

In [8]:
TRAIN_PATH='./DATA/Training/'
folder_paths=os.listdir(TRAIN_PATH)

In [9]:
fashion_df=[]

for folder in folder_paths:
    #print(folder)
    folder_path=TRAIN_PATH+folder
    #print(folder_path)
    json_file=[file for file in os.listdir(folder_path) if file.endswith('.json')]
    #print(json_file)
    for file in json_file:
        file_path=os.path.join(folder_path,file)
        #print(file_path)

        try:
            with open(file_path,'r',encoding='utf-8') as f:
                data=json.load(f)

                if data:
                    for review in data:
                        aspects=pd.json_normalize(review.get('Aspects'))
                    fashion_df.append(aspects)
                else:
                    print(f'No data found in {file}')

        except json.JSONDecodeError:
            print(f"Error loading {file}: Invalid JSON")

In [10]:
if fashion_df:
    train_fashionDF=pd.concat(fashion_df,ignore_index=True)
    print(train_fashionDF)
else:
    print('No Valid Dataframe')

       Aspect              SentimentText SentimentWord SentimentPolarity
0         착용감                     넘 편해서              2                 1
1          소재     니트재질이라보니 보풀감은 어쩔수업머요ㅠㅠ             3                 1
2         활용성                  자주 손가입었어요             2                 1
3         디자인           레깅스에 가깝게보여 좋았는데              3                 1
4         디자인               받고보니 디잔이다릅니다             2                -1
...       ...                        ...           ...               ...
1228      디자인               예쁜 가방도 마련했겠다             3                 1
1229       품질  일단 바퀴의 성능이 탁월해서 너무 좋네요...             6                 1
1230       소재          외관에 스크래치도 덜 가구요..             4                 1
1231       색상                   색상도 좋으네요             2                 1
1232  사용성/편의성       4종류 용도에 맞게 사용하기 편리해요             5                 1

[1233 rows x 4 columns]


In [11]:
val_path='./DATA/Validation/'
folder_paths=os.listdir(val_path)

In [12]:
val_df=[]

for folder in folder_paths:
    path=val_path+folder
    #print(path)

    json_file=[file for file in os.listdir(path) if file.endswith('.json')]

    for file in json_file:
        file_paths=os.path.join(path,file)
        #print(file_paths)

        try:
            with open(file_paths,'r',encoding='utf-8') as f:
                val_data=json.load(f)

                if val_data:
                    for review in val_data:
                        aspects=pd.json_normalize(review.get('Aspects'))
                        val_df.append(aspects)
                else: print(f'no data found in {file}')

        except json.JSONDecodeError: print(f'error loading {file}')

In [13]:
if val_df:
    val_fashionDF=pd.concat(val_df,ignore_index=True)
    print(val_fashionDF)
else: print('no valid dataframe')

      Aspect         SentimentText SentimentWord SentimentPolarity
0         기능          배도 편하게 눌러주고              3                 1
1         길이         미디가 길어 편하다고..             3                 1
2         색상     다섯가지 색상 모두 이쁘네요^^             4                 1
3         기능             입자마자 시원하고             2                 1
4         기능          배 부분도 잘 잡아주고             4                 1
...      ...                   ...           ...               ...
15755     품질                품질 좋아요             2                 1
15756     품질        우선 박음질이 허접합니다.             3                -1
15757     색상              색상은 이뻐요              2                 1
15758     품질  검정백은 지퍼가 뻑뻑해서 잘 안닫혀요             5                -1
15759     가격    50프로 적립 아니였음 반품이였음             4                 1

[15760 rows x 4 columns]


In [14]:
train_fashionDF.isna().sum()

Aspect               0
SentimentText        0
SentimentWord        0
SentimentPolarity    0
dtype: int64

In [15]:
val_fashionDF.isna().sum()

Aspect               0
SentimentText        0
SentimentWord        0
SentimentPolarity    0
dtype: int64

In [16]:
train_fashionDF['Aspect'].value_counts()

Aspect
디자인            131
사이즈            118
가격             107
품질             101
소재              93
기능              90
착화감             83
착용감             69
색상              60
치수/사이즈          51
무게              45
길이              44
두께              37
핏               35
신축성             32
활용성             22
촉감              21
사용성             15
사이즈/폭/길이/두께     14
사용성/편의성         14
기능성             11
마감              10
내구성              8
제품구성             8
냄새               8
굽                5
수납               1
Name: count, dtype: int64

In [17]:
val_fashionDF['Aspect'].value_counts()

Aspect
사이즈            1828
디자인            1421
가격             1282
품질             1213
착용감            1211
기능             1197
소재              966
착화감             867
핏               845
색상              840
신축성             728
치수/사이즈          525
길이              494
무게              484
두께              418
활용성             223
제품구성            200
촉감              186
사용성/편의성         160
사용성             159
기능성             139
사이즈/폭/길이/두께     125
마감               93
내구성              61
수납               40
굽                28
냄새               27
Name: count, dtype: int64

In [18]:
punc=string.punctuation

for p in punc:
    train_fashionDF['SentimentText'] = train_fashionDF['SentimentText'].str.replace(p, '')
    val_fashionDF['SentimentText']=val_fashionDF['SentimentText'].str.replace(p,'')

train_fashionDF

,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,착용감,넘 편해서,2,1
1,소재,니트재질이라보니 보풀감은 어쩔수업머요ㅠㅠ,3,1
2,활용성,자주 손가입었어요,2,1
3,디자인,레깅스에 가깝게보여 좋았는데,3,1
4,디자인,받고보니 디잔이다릅니다,2,-1
...,...,...,...,...
1228,디자인,예쁜 가방도 마련했겠다,3,1
1229,품질,일단 바퀴의 성능이 탁월해서 너무 좋네요,6,1
1230,소재,외관에 스크래치도 덜 가구요,4,1
1231,색상,색상도 좋으네요,2,1


In [19]:
val_fashionDF

,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,기능,배도 편하게 눌러주고,3,1
1,길이,미디가 길어 편하다고,3,1
2,색상,다섯가지 색상 모두 이쁘네요,4,1
3,기능,입자마자 시원하고,2,1
4,기능,배 부분도 잘 잡아주고,4,1
...,...,...,...,...
15755,품질,품질 좋아요,2,1
15756,품질,우선 박음질이 허접합니다,3,-1
15757,색상,색상은 이뻐요,2,1
15758,품질,검정백은 지퍼가 뻑뻑해서 잘 안닫혀요,5,-1


In [20]:
m=re.compile('[^ ㄱ-ㅣ가-힣]+')

train_fashionDF['SentimentText']=train_fashionDF['SentimentText'].apply(lambda x: m.sub(' ', x))
train_fashionDF

,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,착용감,넘 편해서,2,1
1,소재,니트재질이라보니 보풀감은 어쩔수업머요ㅠㅠ,3,1
2,활용성,자주 손가입었어요,2,1
3,디자인,레깅스에 가깝게보여 좋았는데,3,1
4,디자인,받고보니 디잔이다릅니다,2,-1
...,...,...,...,...
1228,디자인,예쁜 가방도 마련했겠다,3,1
1229,품질,일단 바퀴의 성능이 탁월해서 너무 좋네요,6,1
1230,소재,외관에 스크래치도 덜 가구요,4,1
1231,색상,색상도 좋으네요,2,1


In [21]:
val_fashionDF['SentimentText']=val_fashionDF['SentimentText'].apply(lambda x: m.sub(' ', x))
val_fashionDF

,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,기능,배도 편하게 눌러주고,3,1
1,길이,미디가 길어 편하다고,3,1
2,색상,다섯가지 색상 모두 이쁘네요,4,1
3,기능,입자마자 시원하고,2,1
4,기능,배 부분도 잘 잡아주고,4,1
...,...,...,...,...
15755,품질,품질 좋아요,2,1
15756,품질,우선 박음질이 허접합니다,3,-1
15757,색상,색상은 이뻐요,2,1
15758,품질,검정백은 지퍼가 뻑뻑해서 잘 안닫혀요,5,-1


In [22]:
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens

    for token, count in counter.most_common(n_vocab):
        vocab.append(token)

    return vocab

In [24]:
stop_word='./stopwords.txt'

with open(stop_word, 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f]

In [25]:
tokenizer = Okt()

train_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in train_fashionDF['SentimentText']]
val_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in val_fashionDF['SentimentText']]


vocab = build_vocab(corpus=train_tokens, n_vocab=20000, special_tokens=['<pad>','<unk>'])
token_to_id = {token:idx for idx, token in enumerate(vocab)}
id_to_token = {idx:token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '가격', '사이즈', '디자인', '좋아요', '잘', '같아요', '색상', '품질']
1720


In [26]:
len(max(train_tokens, key=len)), len(max(val_tokens, key=len))

(13, 18)

In [27]:
def pad_sequences(sequences, max_length, pad_value):
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [28]:
unk_id = token_to_id["<unk>"]
train_ids = [[token_to_id.get(token, unk_id) for token in review] for review in train_tokens]
val_ids = [[token_to_id.get(token, unk_id) for token in review] for review in val_tokens]

max_length = 18
pad_id = token_to_id['<pad>']
train_ids = pad_sequences(train_ids, max_length, pad_id)
val_ids = pad_sequences(val_ids, max_length, pad_id)

print(train_ids[0])

[ 43 110   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [29]:
print(val_ids[0])

[ 1 21  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
